In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score
from scipy.sparse import hstack

Read Train and Test dataset and fill the missing value with single space. Also, select the comment text from both train and test set and then concatenate them together for the further analysis 

In [2]:
train = pd.read_csv('train.csv').fillna(' ')
test = pd.read_csv('test.csv').fillna(' ')
train_text = train['comment_text']
test_text = test['comment_text']
all_text = pd.concat([train_text, test_text]) # Concatenate

Create binary response: toxic or not

In [24]:
# Create the toxic and not for our prediction response
train['toxic_count'] = [a + b + c + x + y + z for a ,b ,c, x, y, z in 
                        zip(train['toxic'], train['severe_toxic'], train['obscene'],
                           train['threat'], train['insult'], train['identity_hate'])]
train.loc[train['toxic_count'] > 0, 'is_toxic'] = 1
train.loc[train['toxic_count'] == 0, 'is_toxic'] = 0

# Create the toxic and not for our prediction response
test['toxic_count'] = [a + b + c + x + y + z for a ,b ,c, x, y, z in 
                        zip(test['toxic'], test['severe_toxic'], train['obscene'],
                           train['threat'], train['insult'], train['identity_hate'])]
train.loc[train['toxic_count'] > 0, 'is_toxic'] = 1
train.loc[train['toxic_count'] == 0, 'is_toxic'] = 0

In [27]:
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,toxic_count,is_toxic
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,0,0


Use TfidfVectorizer to convert a collection of raw documents to a matrix of TF-IDF features. Term frequency–inverse document frequency (TF-IDF), is a numerical statistic that is intended to reflect how important a word is to a document in a collection or corpus.

In [5]:
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,  # Sublinear scale as the frequency might not be a good indicator
    strip_accents='unicode', # Accents removed
    analyzer='word',
    token_pattern=r'\w{1,}', # cutting words with ' and - in pieces
    ngram_range=(1, 1),
    max_features=500
    )
word_vectorizer.fit(all_text)
train_word_features = word_vectorizer.transform(train_text)
test_word_features = word_vectorizer.transform(test_text)

In [6]:
char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    stop_words='english',
    ngram_range=(2, 6),
    max_features=3000)
char_vectorizer.fit(all_text)
train_char_features = char_vectorizer.transform(train_text)
test_char_features = char_vectorizer.transform(test_text)

In [22]:
train_features = hstack([train_char_features, train_word_features]) # Stack arrays in sequence horizontally
test_features = hstack([test_char_features, test_word_features])

Construct the logistic regression and cross validate it with 5 folds and auc.

In [32]:
train_target = train['is_toxic'].astype('int')
Model = LogisticRegression(solver='sag')
cv_score = np.mean(cross_val_score(Model, train_features, train_target, cv=5, scoring='accuracy'))
print(cv_score)

0.953569256818


Fit our model in testing set and 

In [33]:
test_target = test['is_toxic'].astype('int')
classifier.fit(test_features, test_target)

KeyError: 'is_toxic'

In [29]:
type(train['toxic']

pandas.core.series.Series